In [ ]:
import sys
sys.path.insert(0, './src/')

import nwq2qis
from nwq2qis import *



##
import numpy as np
np.set_printoptions(suppress=True)
import qiskit
import qiskit.quantum_info as qi
from qiskit_nature.second_q.mappers import JordanWignerMapper, InterleavedQubitMapper
from _gcim_utilis import parse_hamiltonian
import scipy.sparse.linalg as ssl
import scipy.linalg as sl
from qiskit_algorithms.optimizers import COBYLA, SPSA
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import EstimatorV2 as Estimator
import json
from qiskit_ibm_runtime import RuntimeEncoder
from qiskit.primitives import StatevectorEstimator
##

n_orb = 6
n_a = 3
n_b = 3
ducc_lvl = 3
results_file = '../Qubit-ADAPT-VQE/printout_qubit_fbp.txt'
output_folder = 'Example_Outputs/FBP' ## Please create this folder before you run the code
noise_factors = [1,3,5,7,9]
selected_iter = 2 ## which qubit-adapt-vqe iteration, start from 0
FIRST_NTERMS = 59  # first 59 groups of Paulis

mol_name =  f'FreeBasedPorphyrin{n_a+n_b}{n_orb}DUCC{ducc_lvl}'
file_path = "../FBP/FreeBasedPorphyrin.out-xacc"
print("  - Number of electrons:", n_a + n_b)
print("  - Number of orbitals:", n_orb)
print("  - DUCC:", ducc_lvl)
print("  - Target molecule:", mol_name)
print("  - Input file:", file_path)

fermi_ham = parse_hamiltonian(file_path, n_orb, interleaved=False, return_type = 'qiskit')
nwqsim_dict = extract_nwqsim_output(results_file, inverse_pauli=False)

mapper = JordanWignerMapper()
qubit_op = mapper.map(fermi_ham)

true_eigvals = ssl.eigsh(qubit_op.to_matrix(sparse=True),2,which='SA')[0]

errors = np.abs( np.array(nwqsim_dict['energies']) - true_eigvals[0] )
for i, error in enumerate(errors):
    if i > 8:
        break
    print(f"{i}:  {str(nwqsim_dict['operators'][i].paulis[0])}   {error}")

  - Number of electrons: 6
  - Number of orbitals: 6
  - DUCC: 3
  - Target molecule: FreeBasedPorphyrin66DUCC3
  - Input file: ../DUCC-Hamiltonian-Library/FBP/FreeBasedPorphyrin.out-xacc
0:  IXYXXIIIIIII   0.0123183193644536
1:  IIIIIIIXYXXI   0.009412132266220397
2:  IIIIIIIYZZZX   0.009143387921540125
3:  IYZZZXIIIIII   0.008927078743795391
4:  IXZZXIIYZZXI   0.0071785118115030855
5:  IIXXIIIIXYII   0.006601006552841682
6:  IIIIIIIYZZZX   0.005938926870157957
7:  IIXZXIIIYZXI   0.00547883963622553
8:  IXZXIIIXZYII   0.004487241603783332


In [44]:
len(qubit_op.group_commuting(qubit_wise=True))

210

## CHOP Hamiltonian

In [ ]:
np.set_printoptions(suppress=True)
pauli_dict, coeff_dict = grouped_pauli_in_dict(qubit_op, use_qwc=True)
pas_ch, cas_ch = pauli_coeff_separation(qubit_op)

ham_dict_ch = { pp:cc for pp,cc in zip(pas_ch, cas_ch)}

test = SparsePauliOp(list(ham_dict_ch.keys()), list(ham_dict_ch.values()) )
test.equiv(qubit_op)

True

## Optimize Parameters

In [ ]:
import nwq2qis
import importlib
importlib.reload(nwq2qis)
from nwq2qis import *

vqe_circ, symbol_params = create_vqe_circ(nwqsim_dict['operators'][:selected_iter+1], 
                                            n_orb*2, n_a+n_b, interleaved=False, half_barriar=True)

## Optimize parameters
def cost_func(parameters):
    """Compute the energy for given parameters."""
    estimator = StatevectorEstimator()
    job = estimator.run([(vqe_circ, qubit_op, parameters)])
    return float( job.result()[0].data.evs )

num_vars = len(nwqsim_dict['parameters'][:selected_iter+1])
optimizer = COBYLA(maxiter=5000, disp=True)
result = optimizer.minimize(cost_func, nwqsim_dict['parameters'][:selected_iter+1])

pre_opt_params = result.x

basis_gates = ['u', 'cx']
trans_vqe_opt = transpile(vqe_circ.assign_parameters(pre_opt_params), basis_gates=basis_gates, 
                          optimization_level=2, seed_transpiler=7)
print(trans_vqe_opt.count_ops(), cost_func(pre_opt_params), nwqsim_dict['energies'][selected_iter])

np.save(f"{output_folder}/optparam-vqe-itr{selected_iter}.npy", pre_opt_params)
pre_opt_params


   Normal return from subroutine COBYLA

   NFVALS =   86   F =-9.867641E+02    MAXCV = 0.000000E+00
   X =-5.764837E-01  -5.767715E-01   5.353437E-02
OrderedDict({'u': 28, 'cx': 19, 'barrier': 1}) -986.7641040677287 -986.7641018705452


array([-0.57648369, -0.57677154,  0.05353437])

In [8]:
vqe_circ.decompose().draw()

┌───────────┐                                                        ░ »
 q_0: ┤ U3(π,0,π) ├────────────────────────────────────────────────────────░─»
      ├───────────┤                                                        ░ »
 q_1: ┤ U3(π,0,π) ├────────────────────────────────────────────────────────░─»
      ├───────────┤                                                        ░ »
 q_2: ┤ U3(π,0,π) ├────────────────────────────────────────────────────────░─»
      └───────────┘                                                        ░ »
 q_3: ─────────────────────────────────────────────────────────────────────░─»
                                                                           ░ »
 q_4: ─────────────────────────────────────────────────────────────────────░─»
                                                                           ░ »
 q_5: ─────────────────────────────────────────────────────────────────────░─»
      ┌───────────┐                                                        ░ »
 q_6: ┤ U3(π,0,π) ├────────────────────────────────────────────────────────░─»
      ├───────────┤┌───┐     ┌───┐┌───────────────┐┌───┐┌───┐              ░ »
 q_7: ┤ U3(π,0,π) ├┤ H ├─────┤ X ├┤ Rz(0.25*θ[0]) ├┤ X ├┤ H ├──────────────░─»
      ├───────────┤├───┤┌───┐└─┬─┘└───────────────┘└─┬─┘├───┤┌───┐         ░ »
 q_8: ┤ U3(π,0,π) ├┤ H ├┤ X ├──■─────────────────────■──┤ X ├┤ H ├─────────░─»
      └───┬────┬──┘├───┤└─┬─┘                           └─┬─┘├───┤┌──────┐ ░ »
 q_9: ────┤ √X ├───┤ X ├──■───────────────────────────────■──┤ X ├┤ √Xdg ├─░─»
          ├───┬┘   └─┬─┘                                     └─┬─┘└┬───┬─┘ ░ »
q_10: ────┤ H ├──────■─────────────────────────────────────────■───┤ H ├───░─»
          └───┘                                                    └───┘   ░ »
q_11: ─────────────────────────────────────────────────────────────────────░─»
                                                                           ░ »
«      ┌───┐                                                                   »
« q_0: ┤ H ├───────────────────────────────────────────────────────────────────»
«      ├───┤           ┌───┐┌───────────────┐┌───┐┌───┐                        »
« q_1: ┤ H ├───────────┤ X ├┤ Rz(0.25*θ[1]) ├┤ X ├┤ H ├────────────────────────»
«      ├───┤      ┌───┐└─┬─┘└───────────────┘└─┬─┘├───┤┌───┐                   »
« q_2: ┤ H ├──────┤ X ├──■─────────────────────■──┤ X ├┤ H ├───────────────────»
«      ├───┴┐┌───┐└─┬─┘                           └─┬─┘├───┤┌──────┐      ┌───┐»
« q_3: ┤ √X ├┤ X ├──■───────────────────────────────■──┤ X ├┤ √Xdg ├──────┤ X ├»
«      ├───┬┘└─┬─┘                                     └─┬─┘└┬───┬─┘┌────┐└─┬─┘»
« q_4: ┤ H ├───■─────────────────────────────────────────■───┤ H ├──┤ √X ├──■──»
«      └───┘                                                 └───┘  └────┘     »
« q_5: ────────────────────────────────────────────────────────────────────────»
«                                                                              »
« q_6: ────────────────────────────────────────────────────────────────────────»
«                                                                              »
« q_7: ────────────────────────────────────────────────────────────────────────»
«                                                                              »
« q_8: ────────────────────────────────────────────────────────────────────────»
«                                                                              »
« q_9: ────────────────────────────────────────────────────────────────────────»
«                                                                              »
«q_10: ────────────────────────────────────────────────────────────────────────»
«                                                                              »
«q_11: ────────────────────────────────────────────────────────────────────────»
«                                                                              »
«     

## Choose First X groups that has the largest sum of coefficients

In [ ]:
from nwq2qis import *

FIRST_NTERMS = 59
## Select first N Groups depends on the sum of abs of coefficients
coeffs_selected = dict_sort_and_select(coeff_dict,FIRST_NTERMS)
pauli_selected = {k:v for k,v in pauli_dict.items() if k in coeffs_selected.keys()}
## Construct Partial Hamiltonian
counter = 0
partial_hamop = None
for k in pauli_selected.keys():
    sub_ham = qiskit.quantum_info.SparsePauliOp(pauli_selected[k], coeffs_selected[k])
    if partial_hamop is None:
        partial_hamop = sub_ham
    else:
        partial_hamop += sub_ham
    counter += 1

## Eigenvalues
partial_eigvals = ssl.eigsh(partial_hamop.to_matrix(sparse=True),2,which='SA')[0]

print("Selected Pauli strings:", len(pauli_selected))
partial_eigvals, abs(partial_eigvals[0] - true_eigvals[0])

Selected Pauli strings: 59


(array([-986.76734685, -986.69267557]), 0.005898407340055201)

### Group Pauli Operators and Generate Circuits

In [ ]:
pauli_dict_partial, coeff_dict_partial = pauli_selected,coeffs_selected
circ_dict = commuting_circs(list(pauli_dict_partial.keys()), trans_vqe_opt, add_measure = True, 
                            noise_factors=noise_factors, inverse_pauli = False, noise_type='half') ## Form circuits for each commuting group
circ_dict_nomeasure = commuting_circs(list(pauli_dict_partial.keys()), trans_vqe_opt, add_measure = False, 
                            noise_factors=noise_factors, inverse_pauli = False, noise_type='half') ## Form circuits for each commuting group
print("Number of commuting groups:",len(list(pauli_dict_partial.keys())))

## Exact Solutions
theo_state = qi.Statevector.from_instruction(vqe_circ.assign_parameters(pre_opt_params)) ## exact state
theo_energy = theo_state.expectation_value(partial_hamop).real
print(theo_energy, partial_eigvals[0], abs(theo_energy - partial_eigvals[0]))

Number of commuting groups: 59
-986.7608388335083 -986.7673468511267 0.006508017618443773


In [45]:
print(f"Chopped/unchopped # of Pauli strings  = {len(partial_hamop)}/{len(qubit_op)}")

Chopped/unchopped # of Pauli strings  = 311/735


## Qiskit Estimator

In [13]:
from qiskit.primitives import StatevectorEstimator
estimator = StatevectorEstimator()
job = estimator.run([(trans_vqe_opt, partial_hamop)])
job_pre = estimator.run([(trans_vqe_opt, partial_hamop)], precision=1e-8)
float( job.result()[0].data.evs ), float( job_pre.result()[0].data.evs ), true_eigvals[0]-float( job.result()[0].data.evs ), true_eigvals[0]-float( job_pre.result()[0].data.evs ), errors[selected_iter]

(-986.7608388335102,
 -986.7608388343695,
 -0.012406424956566298,
 -0.012406424097321178,
 0.009143387921540125)

In [ ]:
trans_vqe_opt.count_ops()

FBPChop_Outputs_QESEM/itr2_qasm


OrderedDict([('u', 28), ('cx', 19), ('barrier', 1)])

## QESEM

In [41]:
exp_obs = partial_hamop
state_prep_circ = trans_vqe_opt

from qiskit.primitives import StatevectorEstimator
estimator = StatevectorEstimator()
rt_exp = estimator.run([(state_prep_circ, exp_obs)]).result()[0].data.evs
float( rt_exp)

-986.7608388335102

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='',
    token=''
)
print(service.backends()) ## service.backends(simulator=False, operational=True, min_num_qubits=5)
backend = service.backend("ibm_kingston")
print(backend.name)

[<IBMBackend('ibm_kingston')>, <IBMBackend('ibm_marrakesh')>, <IBMBackend('ibm_strasbourg')>, <IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_brussels')>, <IBMBackend('ibm_sherbrooke')>, <IBMBackend('ibm_torino')>, <IBMBackend('ibm_aachen')>, <IBMBackend('ibm_fez')>]
ibm_kingston


In [49]:
from qiskit_ibm_catalog import QiskitFunctionsCatalog
catalog = QiskitFunctionsCatalog()
print(catalog.list())
qesem_function = catalog.load("qedma/qesem")

[QiskitFunction(qedma/qesem), QiskitFunction(ibm/circuit-function)]


#### Estimate running time

#### Empirical

In [50]:
state_prep_circ.count_ops(),len(exp_obs)

(OrderedDict([('u', 28), ('cx', 19), ('barrier', 1)]), 311)

In [ ]:
time_estimation_job = qesem_function.run(
    pubs=[(state_prep_circ, [exp_obs])],
    options={
        "default_precision": 0.02, 
        "max_execution_time": 300, 
        "transpilation_level": 1,
        "execution_mode": "batch",  ## "session" / "batch", when batch, QPU is released during classical computation
        "estimate_time_only": "empirical",
    },
    instance='',
    backend_name=backend.name,  # E.g. "ibm_brisbane"
)

print("Time estimation")
print(f">>> Job ID: {time_estimation_job.job_id}")
print(f">>> Job Status: {time_estimation_job.status()}")

Time estimation
>>> Job ID: b730f338-ec04-4704-873f-54463a517cd3
>>> Job Status: QUEUED


In [75]:
time_estimation_job = catalog.get_job_by_id("b730f338-ec04-4704-873f-54463a517cd3")
time_estimation_job.status()

'DONE'

In [ ]:
time_result = time_estimation_job.result()
time_result

PrimitiveResult([PubResult(data=DataBin(), metadata={'time_estimation_sec': 600})], metadata={})

## Execution

In [ ]:
exec_job = qesem_function.run(
    pubs=[(state_prep_circ, [exp_obs])],
    options={
        "default_precision": 0.02, 
        "max_execution_time": 2400, 
        "transpilation_level": 1,
        "execution_mode": "batch",  ## "session" / "batch", when batch, QPU is released during classical computation
    },
    instance='',
    backend_name=backend.name,  # E.g. "ibm_brisbane"
)

print("Execution")
print(f">>> Job ID: {exec_job.job_id}")  ## 4e54cb03-2e84-4efa-ba7d-801877b6e91e
print(f">>> Job Status: {exec_job.status()}")

Execution
>>> Job ID: 4e54cb03-2e84-4efa-ba7d-801877b6e91e
>>> Job Status: QUEUED


In [81]:
exec_job.status()

'DONE'

In [82]:
exec_result = catalog.get_job_by_id("4e54cb03-2e84-4efa-ba7d-801877b6e91e").result()
exec_result

PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(1,), dtype=float64>), stds=np.ndarray(<shape=(1,), dtype=float64>), shape=(1,)), metadata={'gate_fidelities': {'CZ': 0.995571968978783, 'ID1Q': 0.998795695388821}, 'total_shots': 636200, 'mitigation_shots': 300000, 'transpiled_circs': [{'circuit': 'OPENQASM 3.0;\ninclude "stdgates.inc";\nbit[156] c0;\nqubit[156] q1;\nrz(-pi) q1[0];\nrz(-pi) q1[1];\nrz(pi/4) q1[2];\nrz(0) q1[3];\nrz(-pi) q1[4];\nrz(pi/2) q1[64];\nrz(-pi) q1[134];\nrz(pi/4) q1[135];\nrz(-pi) q1[139];\nrz(-pi) q1[155];\nrz(pi/2) q1[109];\nsx q1[0];\nsx q1[1];\nsx q1[2];\nsx q1[3];\nsx q1[4];\nsx q1[64];\nsx q1[134];\nsx q1[135];\nsx q1[139];\nsx q1[155];\nsx q1[109];\nrz(-pi/2) q1[0];\nrz(-pi/2) q1[1];\nrz(0) q1[2];\nrz(-pi/2) q1[3];\nrz(-pi/2) q1[4];\nrz(0) q1[64];\nrz(-pi/2) q1[134];\nrz(0) q1[135];\nrz(-pi/2) q1[139];\nrz(-pi/2) q1[155];\nrz(-pi/2) q1[109];\nsx q1[0];\nsx q1[1];\nsx q1[2];\nsx q1[3];\nsx q1[4];\nsx q1[64];\nsx q1[134];\nsx q1[135];\nsx q1[1

In [83]:
exec_result[0].metadata

{'gate_fidelities': {'CZ': 0.995571968978783, 'ID1Q': 0.998795695388821},
 'total_shots': 636200,
 'mitigation_shots': 300000,
 'transpiled_circs': [{'circuit': 'OPENQASM 3.0;\ninclude "stdgates.inc";\nbit[156] c0;\nqubit[156] q1;\nrz(-pi) q1[0];\nrz(-pi) q1[1];\nrz(pi/4) q1[2];\nrz(0) q1[3];\nrz(-pi) q1[4];\nrz(pi/2) q1[64];\nrz(-pi) q1[134];\nrz(pi/4) q1[135];\nrz(-pi) q1[139];\nrz(-pi) q1[155];\nrz(pi/2) q1[109];\nsx q1[0];\nsx q1[1];\nsx q1[2];\nsx q1[3];\nsx q1[4];\nsx q1[64];\nsx q1[134];\nsx q1[135];\nsx q1[139];\nsx q1[155];\nsx q1[109];\nrz(-pi/2) q1[0];\nrz(-pi/2) q1[1];\nrz(0) q1[2];\nrz(-pi/2) q1[3];\nrz(-pi/2) q1[4];\nrz(0) q1[64];\nrz(-pi/2) q1[134];\nrz(0) q1[135];\nrz(-pi/2) q1[139];\nrz(-pi/2) q1[155];\nrz(-pi/2) q1[109];\nsx q1[0];\nsx q1[1];\nsx q1[2];\nsx q1[3];\nsx q1[4];\nsx q1[64];\nsx q1[134];\nsx q1[135];\nsx q1[139];\nsx q1[155];\nsx q1[109];\nrz(1.4711276743037338) q1[0];\nrz(pi/2) q1[1];\nrz(3*pi/4) q1[2];\nrz(-pi) q1[3];\nrz(pi/2) q1[4];\nrz(pi/2) q1[64];\n

In [89]:
obs_ps, obs_cs = pauli_coeff_separation(exp_obs)
obs_dict = { pp:cc for pp,cc in zip(obs_ps, obs_cs)}
zero_coeff = obs_dict['I'*12]
full_qesem_exp = exec_result[0].data['evs'][0]

print(theo_energy, partial_eigvals[0], abs(theo_energy - partial_eigvals[0]))
print(zero_coeff, full_qesem_exp, exec_result[0].data['stds'][0])
print(abs(full_qesem_exp - theo_energy), abs(full_qesem_exp - partial_eigvals[0]), abs(full_qesem_exp - true_eigvals[0]))

-986.7608388335083 -986.7673468511267 0.006508017618443773
-985.5993857093426 -986.7369169524959 0.017124152214938404
0.02392188101237025 0.030429898630814023 0.036328305970869224


In [92]:
state_prep_circ.draw()

global phase: 3.259
          ┌──────────┐                                                        »
 q_0: ────┤ U(π,0,π) ├────────────────────────────────────────────────────────»
          ├──────────┤                                                        »
 q_1: ────┤ U(π,0,π) ├────────────────────────────────────────────────────────»
          ├──────────┤                                                        »
 q_2: ────┤ U(π,0,π) ├────────────────────────────────────────────────────────»
          └──────────┘                                                        »
 q_3: ────────────────────────────────────────────────────────────────────────»
                                                                              »
 q_4: ────────────────────────────────────────────────────────────────────────»
                                                                              »
 q_5: ────────────────────────────────────────────────────────────────────────»
          ┌──────────┐                                                        »
 q_6: ────┤ U(π,0,π) ├────────────────────────────────────────────────────────»
        ┌─┴──────────┴─┐           ┌───┐┌─────────────────┐┌───┐┌────────────┐»
 q_7: ──┤ U(π/2,-π,-π) ├───────────┤ X ├┤ U(0,0,-0.14412) ├┤ X ├┤ U(π/2,0,π) ├»
        ├──────────────┤      ┌───┐└─┬─┘└─────────────────┘└─┬─┘└───┬───┬────┘»
 q_8: ──┤ U(π/2,-π,-π) ├──────┤ X ├──■───────────────────────■──────┤ X ├─────»
      ┌─┴──────────────┴┐┌───┐└─┬─┘                                 └─┬─┘     »
 q_9: ┤ U(π/2,-π/2,π/2) ├┤ X ├──■─────────────────────────────────────■───────»
      └──┬────────────┬─┘└─┬─┘                                                »
q_10: ───┤ U(π/2,0,π) ├────■──────────────────────────────────────────────────»
         └────────────┘                                                       »
q_11: ────────────────────────────────────────────────────────────────────────»
                                                                              »
«                                         ░    ┌────────────┐       »
« q_0: ───────────────────────────────────░────┤ U(π/2,0,π) ├───────»
«                                         ░    ├────────────┤       »
« q_1: ───────────────────────────────────░────┤ U(π/2,0,π) ├───────»
«                                         ░    ├────────────┤       »
« q_2: ───────────────────────────────────░────┤ U(π/2,0,π) ├───────»
«                                         ░ ┌──┴────────────┴─┐┌───┐»
« q_3: ───────────────────────────────────░─┤ U(π/2,-π/2,π/2) ├┤ X ├»
«                                         ░ └──┬────────────┬─┘└─┬─┘»
« q_4: ───────────────────────────────────░────┤ U(π/2,0,π) ├────■──»
«                                         ░    └────────────┘       »
« q_5: ───────────────────────────────────░─────────────────────────»
«                                         ░                         »
« q_6: ───────────────────────────────────░─────────────────────────»
«                                         ░                         »
« q_7: ───────────────────────────────────░─────────────────────────»
«      ┌────────────┐                     ░                         »
« q_8: ┤ U(π/2,0,π) ├─────────────────────░─────────────────────────»
«      └───┬───┬────┘┌──────────────────┐ ░                         »
« q_9: ────┤ X ├─────┤ U(-π/2,-π/2,π/2) ├─░─────────────────────────»
«          └─┬─┘     └──┬────────────┬──┘ ░                         »
«q_10: ──────■──────────┤ U(π/2,0,π) ├────░─────────────────────────»
«                       └────────────┘    ░                         »
«q_11: ───────────────────────────────────░─────────────────────────»
«                                         ░                         »
«                                                                      »
« q_0: ────────────────────────────────────────────────────────────────»
«                           ┌───┐┌─────────────────┐┌───┐┌────────────┐»
« q_1: ────

In [98]:
vqe_circ.decompose().count_ops(), transpile(vqe_circ, optimization_level=2).count_ops()

(OrderedDict([('cx', 20),
              ('h', 14),
              ('u3', 6),
              ('sx', 3),
              ('rz', 3),
              ('sxdg', 3),
              ('barrier', 1)]),
 OrderedDict([('cx', 18),
              ('h', 11),
              ('x', 4),
              ('rz', 3),
              ('u2', 2),
              ('sx', 2),
              ('sxdg', 2),
              ('barrier', 1),
              ('unitary', 1)]))